## Model 1: Entry and exit rules for backtesting

**Model 1:** EMA50 crossover entry and EMA20 downtrend signal exit.

In [1]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import sys
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEG import main


In [2]:
# Call the main function and store the returned DataFrame
df = main()

File loaded successfully!


In [3]:
# Show dataframe
print(df.head())

       symbol  close   open   high    low   volume   timestamp  \
318      AAOI   5.27   5.20   5.30   5.08   478600  2022-01-03   
507822   PACB  20.90  20.50  21.01  19.51  2428600  2022-01-03   
505755    OSW  10.50  10.08  10.73  10.08   252000  2022-01-03   
505066   OSPN  17.23  16.99  17.26  16.74   126700  2022-01-03   
503688   ORLA   3.88   3.81   3.88   3.64    43300  2022-01-03   

        exceptionalCandleSize  ema20  ema50  openHigher  averageVolume  \
318                         0   5.27   5.27           0            NaN   
507822                      0  20.90  20.90           0            NaN   
505755                      0  10.50  10.50           0            NaN   
505066                      0  17.23  17.23           0            NaN   
503688                      0   3.88   3.88           0            NaN   

        strongVolume  strongVolume6MoPrior  accVolume  uptrend  closeHigh  \
318                0                     1          0        0          0   
5078

In [5]:
# Example DataFrame setup
df_test = pd.DataFrame({
    'timestamp': pd.to_datetime([1,2,3,4,5,6,7,8,9,10], unit='D', origin='2022-01-01'),  # sample dates
    'symbol': ['AA','AA','AA','AA','AA','AA','AA','AA','AA','AA'],  
    'close': [10,11,11,10.2,11,13,12,14,13,15],
    'ema50': [9,10,10.5,10.5,10.5,11,11.5,11.5,12,13],
    'pegAlert': [0,1,0,0,0,0,0,0,0,0],  
})

In [10]:

# Function to apply the entry rules per symbol
def calculate_entry(df_test):
    df_test['entry'] = 0  # Initialize the 'entry' column
    
    # Loop through the DataFrame row by row
    for i in range(len(df_test)):
        if df_test.loc[i, 'pegAlert'] == 1:
            # Find the next 10 days (starting from the next day, i+1)
            for j in range(i + 1, min(i + 8, len(df_test))):
                if df_test.loc[j, 'close'] <= df_test.loc[j, 'ema50']:
                    df_test.loc[i, 'entry'] = 1
                    break  # Stop once the condition is met
    return df_test

# Group by symbol and apply the function
df_test_grouped = df_test.groupby('symbol').apply(calculate_entry).reset_index(drop=True)

# Display the updated DataFrame
print(df_test_grouped)


   timestamp symbol  close  ema50  pegAlert  entry
0 2022-01-02     AA   10.0    9.0         0      0
1 2022-01-03     AA   11.0   10.0         1      1
2 2022-01-04     AA   11.0   10.5         0      0
3 2022-01-05     AA   10.2   10.5         0      0
4 2022-01-06     AA   11.0   10.5         0      0
5 2022-01-07     AA   13.0   11.0         0      0
6 2022-01-08     AA   12.0   11.5         0      0
7 2022-01-09     AA   14.0   11.5         0      0
8 2022-01-10     AA   13.0   12.0         0      0
9 2022-01-11     AA   15.0   13.0         0      0


C:\Users\SamuliMustonen\AppData\Local\Temp\ipykernel_42628\2047722309.py:16: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_test_grouped = df_test.groupby('symbol').apply(calculate_entry).reset_index(drop=True)
